<a href="https://colab.research.google.com/github/diego-andrade279/Aula-de-deploy-de-modelos-de-machine-learning-usando-streamlit/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning (Limpeza de Dados)

In [ ]:
import pandas as pd

# configuração para melhorar a exibição dos dados
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Etapa 1 - Carregamento dos dados

In [ ]:
df = pd.read_csv('http://www.edsonmelo.com.br/datasets/dados_alunos.csv')

### Etapa 2 - Analisando o DataFrame

In [ ]:
df.head()

In [ ]:
df.info()

### Etapa 3 - Verificando dados nulos `NaN`

Analisando os dados acima, vemos que o campo `estrutura` possui valores nulos.

No primeiro momento podemos pensar em excluir esses dados, mas poderia interferir em análises posteriores.

Por esse motivo vamos preencher os dados de `modalidade` com _`Genérico-Genérica`_.

In [ ]:
df[df.modalidade.isnull()]

In [ ]:
# Preenchendo com valores padrão o campo com valores NaN
df.loc[df.modalidade.isnull(),'modalidade']='Genérico-Genérica'
df.info()

### Etapa 4 - Análise da Cardinalidade

Podemos verificar que a cardinalidade é alta, o que exige tratamento dos dados. Como em todo trabalho de Ciência de Dados, é preciso conhecer o que representa o `dataset` e seus campos. A seguir é descrita a representação de cada coluna.

- `identificador`
        - Matricula
        - Campus em que está matriculado
            - VG: Vergueiro
            - VM: Vila Maria
            - VP: Vila Prudente
            - SA: Santo Amaro
            - MM: Memorial
        - Ano de ingresso na universidade
        
- `cursosem`
    - Nome do curso em que está matriculado
        - TADS: Tecnologia em Análise e Desenvolvimento de Sistemas
        - TRC: Tecnologia em Redes de Computadores
        - TBD: Tecnologia em Banco de Dados
        - TGTI: Tecnologia em Gestão da Tecnologia da Informação
        - CC: Bacharelado em Ciências da Computação
        - SI: Bacharelado em Sistemas de Informação
    - Semestre que está cursando
- `modalidade`:  tipo do curso em que está matriculado
    - Presencial (PRES)
        - Mod: Modular
        - Seq: Sequencial
     - EaD (A Distância)
        - Mod: Modular
        - Seq: Sequencial
- `bolsista`: se possui algum tipo de bolsa de estudos
- `chipvivo`: se retirou o chip para acesso ao Telepresencial

In [ ]:
# analisando a cardinalidade dos dados utilizando o campo chipvivo como parâmetro
df.groupby(["chipvivo","identificador"])["chipvivo"].count()

### Etapa 5 - Iniciando o tratamento dos dados

Não existe uma regra definida por onde começar o tratamento de dados. Entretanto, é indicado que inicialmente seja, procurados padrões nos dados. Isso pode ser feito de maneira intutiva ou então com o auxílio de algum código. Vamos iniciar nosso estudo realizando a separação dos dados que estão agregados nas colunas. Para isso podemos usar o fatiamento de listas.

In [ ]:
# Primeiro vamos verificar os dados apenas da coluna 'indetificador' para analisar o padrão
df.identificador.sample(10)

In [ ]:
# Pegando a primeira linha para testar
df.identificador.head(1)

In [ ]:
# Analisando os valores acima é possível determinar que os dados estão dividos em posições determinadas pelo tamanho. Vejamos:

valor = '68C2A2643CSA2019'
print('Matrícula:', valor[0:10]) # lembre-se que o limite suprior não é inclusivo (não conta)
print('Campus:', valor[10:12])
print('Ingresso:', valor[-4:])

In [ ]:
# Agora que identificamos o padrão, vamos criar novas colunas com os dados separados
df['matricula'] = df.identificador.str[0:10]
df['campus'] = df.identificador.str[10:12]
df['ingresso'] = df.identificador.str[-4:]

In [ ]:
# Vamos verificar a cardinalidade novamente usando "campus" para saber os alunos que retiraram o chip
# Perceba que MM é o campus com mais alunos que retiraram o chip.
# Isso não era possível antes da separaçao dos dados
df.groupby(["chipvivo","campus"])["chipvivo"].count()

In [ ]:
# Vamos agora separar o campo "curso" usando a mesma lógica anterior.
# Como o último valor é um número, isso indica que é o semestre.
# Cuidado com o nome do campo para não sobrepor os dados.
df['curso'] = df.cursosem.str[:-1]
df['semestre'] = df.cursosem.str[-1:]
df.head()

In [ ]:
# Dando continuidade, vamos separar a coluna modalidade. Lembre-se que preenchemos anteriormente os valores ausentes
# Perceba que existe um delimitador nesta coluna, o hífen (-). Então vamos usar outro mátodo para realizar a separação: o split()
# Dados da primeira linha: PRES-Mod
valor = 'PRES-Mod'
valor.split('-')

In [ ]:
# Foi criada uma lista e vamos recuperar os valores pelos índices
print(valor.split('-')[0])
print(valor.split('-')[1])

In [ ]:
# O valor retornado está abreviado, então vamos aplicar uma expressão regular para substituir o valor por extenso

import re # biblioteca para expressões regulares

# separando os valores
df['tipo'] = df.modalidade.str.split('-').str.get(0)
df['estrutura'] = df.modalidade.str.split('-').str.get(1)

# usando um lambda para percorrer as linhas e fazer a atualização dos dados
df['tipo'] = df.tipo.apply(lambda x: re.sub('PRES', 'Presencial', str(x)))
df.head()

In [ ]:
# Outra forma de renomear valores, utilizando funções do Pandas
df['estrutura'] = df['estrutura'].replace(['Mod','Seq'],['Modular','Sequencial'])
df.head()

In [ ]:
# Copiando o DataFrame e excluindo as colunas antigas
df.drop(['identificador', 'cursosem', 'modalidade'], axis=1, inplace=True)
df.head()

In [ ]:
# Renomeando colunas
# Neste caso vamos usar esse método para relembrar como renomear colunas.
# Além disso, no passo anterior foi definido o nome como 'tipo' porque já tinha a 'modalidade'
colunas = {'tipo':'modalidade'}
df.rename(columns=colunas, inplace=True)
df.head()

In [ ]:
# Reordenando as colunas
df = df.reindex(columns=['matricula','bolsista','ingresso','campus','curso','semestre','modalidade','estrutura','chipvivo'])
df.head()

In [ ]:
df.info()

### Etapa 6 - Alterando `tipos` de dados das colunas

In [ ]:
# O tipo de dados de 'semestre' está como String. Vamos trocar para 'int'
# Apesar do campo 'ingresso' representar o ano, não há necessidade conversão para o tipo data. Vamos deixar como 'int'.
df['ingresso'] = df['ingresso'].astype(int)
df['semestre'] = df["semestre"].astype(int)

In [ ]:
df.info()

### Etapa 7 - Salvando os dados tratados

In [ ]:
# Agora que já temos o DataFrame totalmente tratado, vamos salvar uma cópia.
df.to_csv('dados_limpos.csv', index=None)

In [ ]:
df.head()

### Etapa 8 - Realizando algumas análises

In [ ]:
# Número de ingressos por ano
df.groupby(["ingresso"])["ingresso"].count().plot(kind='bar')

In [ ]:
# Número de alunos por campus
df.groupby(['campus'])['campus'].count().sort_values(ascending=False)

In [ ]:
# Gráfico em ordem decrescente
df.groupby(['campus'])['campus'].count().sort_values(ascending=False).plot(kind='bar')